In [85]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime
from sklearn.ensemble import RandomForestRegressor
sns.set()

import statsmodels.api as sm
%matplotlib inline

# 設定
import warnings
warnings.filterwarnings("ignore")

from fbprophet import Prophet

In [86]:
item=pd.read_csv("/Users/tasuke0630/making/DataScience/kaggle/future_sales/items.csv")
item_categories=pd.read_csv("/Users/tasuke0630/making/DataScience/kaggle/future_sales/item_categories.csv")
sales_train=pd.read_csv("/Users/tasuke0630/making/DataScience/kaggle/future_sales/sales_train.csv")
shops=pd.read_csv("/Users/tasuke0630/making/DataScience/kaggle/future_sales/shops.csv")
test_csv=pd.read_csv("/Users/tasuke0630/making/DataScience/kaggle/future_sales/test.csv")
sample_submission=pd.read_csv("/Users/tasuke0630/making/DataScience/kaggle/future_sales/sample_submission.csv")

In [3]:
# 日付カラムを正しくフォーマットする
sales_train.date=sales_train.date.apply(lambda x:datetime.datetime.strptime(x, '%d.%m.%Y'))
# 確認
print(sales_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   date            datetime64[ns]
 1   date_block_num  int64         
 2   shop_id         int64         
 3   item_id         int64         
 4   item_price      float64       
 5   item_cnt_day    float64       
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 134.4 MB
None


In [32]:
#各月のshopで売れたアイテムの総数
monthly_shop_sales=sales_train.groupby(["date_block_num","shop_id"])["item_cnt_day"].sum()
# get the shops to the columns
monthly_shop_sales=monthly_shop_sales.unstack(level=1)
monthly_shop_sales=monthly_shop_sales.fillna(0)
monthly_shop_sales.index=dates
monthly_shop_sales=monthly_shop_sales.reset_index()
monthly_shop_sales

shop_id,index,0,1,2,3,4,5,6,7,8,...,50,51,52,53,54,55,56,57,58,59
0,2013-01-01,5578.0,2947.0,1146.0,767.0,2114.0,0.0,3686.0,2495.0,1463.0,...,3406.0,2748.0,1812.0,2055.0,9386.0,0.0,3238.0,0.0,0.0,2017.0
1,2013-02-01,6127.0,3364.0,488.0,798.0,2025.0,877.0,4007.0,2513.0,1156.0,...,3054.0,2284.0,1737.0,1906.0,8075.0,0.0,2655.0,0.0,0.0,1897.0
2,2013-03-01,0.0,0.0,753.0,959.0,2060.0,1355.0,4519.0,2460.0,977.0,...,3610.0,2605.0,2034.0,2365.0,9488.0,0.0,3099.0,6586.0,3773.0,2028.0
3,2013-04-01,0.0,0.0,583.0,647.0,285.0,1008.0,3168.0,1540.0,-1.0,...,2740.0,1945.0,1446.0,1515.0,6726.0,0.0,2153.0,5493.0,2831.0,1388.0
4,2013-05-01,0.0,0.0,553.0,710.0,1211.0,1110.0,3022.0,1647.0,0.0,...,2839.0,2243.0,1482.0,1767.0,7006.0,852.0,2140.0,5233.0,2308.0,1374.0
5,2013-06-01,0.0,0.0,832.0,784.0,1464.0,1393.0,3847.0,2085.0,0.0,...,3761.0,2200.0,1815.0,2125.0,8101.0,728.0,3105.0,5987.0,2972.0,1707.0
6,2013-07-01,0.0,0.0,807.0,721.0,1378.0,1265.0,3360.0,2031.0,0.0,...,3235.0,2273.0,1616.0,1975.0,7225.0,795.0,3294.0,5827.0,3122.0,1747.0
7,2013-08-01,0.0,0.0,875.0,890.0,1713.0,1510.0,3702.0,2397.0,0.0,...,3718.0,2142.0,1772.0,2367.0,7792.0,719.0,3311.0,5413.0,2863.0,2048.0
8,2013-09-01,0.0,0.0,945.0,1026.0,1536.0,1298.0,4208.0,2599.0,0.0,...,3655.0,1903.0,2125.0,2440.0,7827.0,812.0,2829.0,5213.0,2735.0,2008.0
9,2013-10-01,0.0,0.0,795.0,793.0,1482.0,1267.0,3675.0,2236.0,0.0,...,3525.0,1809.0,1965.0,2245.0,6676.0,916.0,2375.0,5686.0,3022.0,1751.0


In [56]:
# Calculating the base forecasts using prophet
# From HTSprophet pachage -- https://github.com/CollinRooney12/htsprophet/blob/master/htsprophet/hts.py
forecastsDict = {}
for node in range(60):
    # take the date-column and the col to be forecasted
    nodeToForecast = pd.concat([monthly_shop_sales.iloc[:,0], monthly_shop_sales.iloc[:, node+1]], axis = 1)
#     print(nodeToForecast.head())  # just to check
# rename for prophet compatability
    nodeToForecast = nodeToForecast.rename(columns = {nodeToForecast.columns[0] : 'ds'})
    nodeToForecast = nodeToForecast.rename(columns = {nodeToForecast.columns[1] : 'y'})
    growth = 'linear'
    m = Prophet(growth, yearly_seasonality=True)
    m.fit(nodeToForecast)
    future = m.make_future_dataframe(periods = 1, freq = 'MS')
    print(future)
    forecastsDict[node] = m.predict(future)


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

In [57]:
#predictions = np.zeros([len(forecastsDict[0].yhat),1]) 
nCols = len(list(forecastsDict.keys()))+1
for key in range(0, nCols-1):
    f1 = np.array(forecastsDict[key].yhat)
    f2 = f1[:, np.newaxis]
    if key==0:
        predictions=f2.copy()
       # print(predictions.shape)
    else:
        predictions= np.concatenate((predictions, f2), axis = 1)

In [68]:
x=pd.DataFrame(predictions)
x

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,3912.117261,2060.580211,1130.005652,812.074980,2008.221319,452.744107,3671.375082,2459.153950,1009.517982,-2.083401,...,3516.848376,2656.603194,1875.083766,2045.879013,10352.470224,-168.956669,3333.523450,1718.427626,942.887695,1998.389795
1,4145.889330,2285.427046,593.177881,841.379504,1938.718392,1129.240937,4003.835066,2539.788455,672.466903,96.936694,...,3385.269474,2217.499839,1836.896327,1911.905779,9089.275166,359.627647,2769.232165,2163.725261,1171.474567,1936.569222
2,-770.847043,-417.690880,833.837272,1035.684781,1973.787914,1556.924530,4497.240000,2470.421118,556.070814,158.358337,...,4001.723062,2571.151053,2148.970658,2363.817337,10129.114808,76.244567,3141.354443,7533.521154,4237.309715,2083.352223
3,-313.766306,-170.851165,570.517302,611.885227,498.908669,1050.398086,3360.934530,1589.335738,-123.353966,150.026097,...,3087.352034,2003.960084,1468.565345,1530.799582,7526.099077,-305.391584,2120.330136,5999.952057,3082.159528,1479.584642
4,129.506056,68.942825,557.220162,675.412828,1337.973222,1110.962660,3109.346929,1703.596716,10.043212,104.709990,...,3049.713803,2166.109585,1533.040000,1792.525905,7422.915846,360.625037,2161.036921,5204.786091,2284.154207,1447.330778
5,523.080550,282.187933,794.718064,750.248164,1530.079610,1330.559799,3848.358309,2134.751271,135.554458,26.533558,...,3726.609312,2183.562930,1729.084163,2143.734291,8104.341311,210.177029,3083.159683,5338.621097,2579.736025,1677.361728
6,756.106063,408.853874,736.052458,692.647282,1400.830726,1212.781771,3370.373055,2008.274703,218.120509,-61.887810,...,3109.865353,2279.060628,1559.913528,2004.072504,6907.856027,335.242131,3159.106965,4943.283893,2568.771518,1634.885998
7,776.834050,420.773952,819.082114,869.945826,1744.654081,1418.865530,3632.953792,2390.749169,238.981897,-141.375301,...,3402.303008,2159.494050,1737.604936,2327.388843,7761.608798,494.612813,3373.297020,4462.847357,2496.777779,1874.878712
8,662.339138,359.429690,905.766850,1004.182107,1516.117220,1247.314878,4086.122491,2488.229039,218.227825,-206.270028,...,3197.451592,1918.876154,1947.124895,2385.667839,7594.590640,144.184713,2684.963372,4433.908732,2353.522065,1844.371157
9,364.019535,198.386516,831.808718,799.696658,1468.120142,1232.947157,3567.990030,2186.814089,137.111470,6740.874731,...,2986.462929,1947.366145,1846.848418,2235.689659,6902.631329,1947.337201,2330.347356,5151.559308,2712.226089,1671.974986


In [114]:
#shop0で今まで売れた数
sum_x=0
for i in range(35):
    sum_x+=x.loc[i,0]
    
sum_x

12021.464231710926

In [113]:
#34月目に売れた総数
sum_x1=0
for i in range(60):
    sum_x1+=x.loc[34,i]
    
sum_x1

98561.98256963148

In [77]:
sales_train2=sales_train.copy()
sales_train2.drop(["date","item_price"], axis=1, inplace=True)
#各月で売れたアイテムの数
sales_train2=sales_train2.groupby(["date_block_num","shop_id","item_id"]).sum()
sales_train2=sales_train2.pivot_table(index=["shop_id","item_id"], columns="date_block_num", values="item_cnt_day", fill_value=0)
sales_train2.reset_index(inplace=True)
sales_train2

date_block_num,shop_id,item_id,0,1,2,3,4,5,6,7,...,24,25,26,27,28,29,30,31,32,33
0,0,30,0,31,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,31,0,11,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,32,6,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,33,3,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,35,1,14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424119,59,22154,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
424120,59,22155,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
424121,59,22162,0,0,0,0,0,0,0,0,...,0,9,4,1,1,0,0,1,0,0
424122,59,22164,0,0,0,0,0,0,0,0,...,0,2,1,2,0,0,1,0,0,0


In [87]:
x1=sales_train2.values
x2=sales_train2.keys()

#各ショップですべての月の売れたアイテムごとの総数
item_list=[0*i for i in range(424124)]

for i in range(424124):
    for j in range(34):
        item_list[i]+=x1[i][j+2]
        

In [88]:
#shop_idの数を数える
shop_list=[0*i for i in range(60)]

for i in range(424124):
    for j in range(60):
        shop_list[j]+=np.where(x1[i][0]==j,1,0)

In [111]:
z=[0*i for i in range(60)]

#shop0
for i0 in range(shop_list[0]):
    z[0]+=item_list[i]
    
#shop1
for i1 in range(shop_list[1]):
    i1+=i0+1
    z[1]+=item_list[i1]
    
#shop2
for i2 in range(shop_list[2]):
    i2+=i1+1
    z[2]+=item_list[i2]
    
#shop3
for i3 in range(shop_list[3]):
    i3+=i2+1
    z[3]+=item_list[i3]

In [112]:
z

[111600,
 6311,
 30620,
 28355,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [100]:
item_list

[31,
 11,
 16,
 6,
 15,
 1,
 1,
 1,
 1,
 2,
 5,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 30,
 2,
 1,
 1,
 3,
 1,
 1,
 3,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 5,
 8,
 2,
 1,
 1,
 2,
 1,
 13,
 5,
 1,
 10,
 14,
 6,
 1,
 10,
 1,
 2,
 22,
 4,
 1,
 2,
 8,
 3,
 3,
 2,
 1,
 2,
 2,
 2,
 4,
 1,
 9,
 1,
 1,
 1,
 2,
 1,
 4,
 1,
 2,
 4,
 3,
 1,
 4,
 7,
 9,
 2,
 2,
 2,
 3,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 3,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 3,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 2,
 4,
 10,
 1,
 1,
 3,
 2,
 1,
 1,
 3,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 13,
 7,
 6,
 5,
 1,
 2,
 2,
 11,
 8,
 2,
 3,
 2,
 13,
 11,
 12,
 4,
 13,
 1,
 1,
 1,
 4,
 3,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 4,
 1,
 2,
 5,
 3,
 9,
 1,
 1,
 1,
 8,
 22,
 1,
 15,
 11,
 3,
 2,
 1,
 42,
 10,
 10,
 1,
 1,
 1,
 1,
 3,
 12,
 1,
 1,
 18,
 3,
 11,
 5,
 6,
 1,
 1,
 6,
 6,
 1,
 6,
 3,
 2,
 1,
 6,
 2,
 2,
 3

In [101]:
shop_list

[3600,
 2523,
 4849,
 5261,
 7309,
 7084,
 11856,
 8246,
 1909,
 1404,
 6002,
 371,
 7382,
 4865,
 7451,
 8269,
 9375,
 5158,
 9073,
 9558,
 746,
 10145,
 7951,
 2949,
 8087,
 15367,
 9289,
 10842,
 12572,
 7806,
 9512,
 14416,
 3299,
 1900,
 1860,
 9108,
 258,
 8405,
 7356,
 4096,
 817,
 7227,
 12034,
 6737,
 8894,
 7303,
 9149,
 7912,
 4063,
 3786,
 9176,
 8739,
 8416,
 7455,
 13863,
 1985,
 11049,
 12281,
 8393,
 7336]

In [102]:
len(shop_list)

60